In [1]:
import pandas as pd
import sqlalchemy as sql
import database as db
import betSim as bs
import mysql.connector as mysql

In [2]:
db.getTeamNames('LCK_Summer_2022')

['T1',
 'Gen.G eSports',
 'DRX',
 'Nongshim RedForce',
 'Hanwha Life eSports',
 'KT Rolster',
 'Liiv SANDBOX',
 'Kwangdong Freecs',
 'DWG KIA',
 'Fredit BRION']

In [8]:
# very basic simulation of a beting strategy
units =100.0
for i in range(len(df)):
    if (units < 1):
        print(f'no more units left after {str(i)} games')
        break
    outcome = bs.towerBet(i, df, 'under', 11.5)
    if(outcome == 1):
        units += 0.83
    else:
        units -= 1

print(f'units: {str(units)}')

units: 147.86000000000044


In [5]:
#test if the database is working
db_test = mysql.connect(host='140.238.205.186', user='loki', passwd='Thethethe3!', database='lolz')
mycursor = db_test.cursor()
mycursor.execute('SHOW DATABASES')
print(mycursor.fetchall())

[('information_schema',), ('lolz',), ('mysql',), ('performance_schema',), ('sys',), ('test',)]
